# Data Understanding

### Import Data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

client = 'TOOL_CLIENT.csv'
sales  = 'TOOL_SALES.csv'
joint = 'TOOL_JOINT.csv'

In [19]:
# df_client = pd.read_csv(client)
# df_sales = pd.read_csv(sales)

# df = pd.merge(df_client, df_sales, on='CLIENT_ID')

df = pd.read_csv(joint)
df.head()

,CLIENT_ID,CLIENT_CREATE DATE,REGION,TRADE SECTOR,N_EMPLOYEES,ECONOMIC_POT,ECO_POT_CLASS,RISK_CAT,YYYYMM,ITEM_ID,FLG_TOOL,SALES_CHANNEL,NET,UNIT,FAMILY_CODE,GROUP_CODE,CANCELLED
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,P,XBXV2EB,XBXV2EB0102,NaN
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,P,XBXV2AB,XBXV2AB0102,NaN
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,P,XBXV2EB,XBXV2EB0102,NaN
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,P,XBXV1AF,XBXV1AF0201,NaN
4,939,2005-11-15,LE,15500,2,681.26,E,3d,2017-05-01,8241,0,B,64.51,P,XAES4AA,XAES4AA0101,X


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050449 entries, 0 to 2050448
Data columns (total 17 columns):
 #   Column              Dtype  
---  ------              -----  
 0   CLIENT_ID           int64  
 1   CLIENT_CREATE DATE  object 
 2   REGION              object 
 3   TRADE SECTOR        int64  
 4   N_EMPLOYEES         int64  
 5   ECONOMIC_POT        float64
 6   ECO_POT_CLASS       object 
 7   RISK_CAT            object 
 8   YYYYMM              object 
 9   ITEM_ID             int64  
 10  FLG_TOOL            int64  
 11  SALES_CHANNEL       object 
 12  NET                 float64
 13  UNIT                object 
 14  FAMILY_CODE         object 
 15  GROUP_CODE          object 
 16  CANCELLED           object 
dtypes: float64(2), int64(5), object(10)
memory usage: 265.9+ MB


In [ ]:
# converting date columns to datetime
df['CLIENT_CREATE DATE'] = pd.to_datetime(df['CLIENT_CREATE DATE'])
df['YYYYMM'] = pd.to_datetime(df['YYYYMM'].astype(str), format='%Y-%m-%d')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050449 entries, 0 to 2050448
Data columns (total 17 columns):
 #   Column              Dtype         
---  ------              -----         
 0   CLIENT_ID           int64         
 1   CLIENT_CREATE DATE  datetime64[ns]
 2   REGION              object        
 3   TRADE SECTOR        int64         
 4   N_EMPLOYEES         int64         
 5   ECONOMIC_POT        float64       
 6   ECO_POT_CLASS       object        
 7   RISK_CAT            object        
 8   YYYYMM              datetime64[ns]
 9   ITEM_ID             int64         
 10  FLG_TOOL            int64         
 11  SALES_CHANNEL       object        
 12  NET                 float64       
 13  UNIT                object        
 14  FAMILY_CODE         object        
 15  GROUP_CODE          object        
 16  CANCELLED           object        
dtypes: datetime64[ns](2), float64(2), int64(5), object(8)
memory usage: 265.9+ MB


In [ ]:
# exporting the merged data

# df.to_csv('TOOL_JOINT.csv', index=False)

Create a new variable to identify client

Create a new variable to identify unique sales

In [ ]:
# creating a new column for identifie the unique sales
df['SALES_ID'] = df['ECONOMIC_POT'].astype(str) + '_' + df['YYYYMM'].dt.strftime('%Y%m')

df.head()

,CLIENT_ID,CLIENT_CREATE DATE,REGION,TRADE SECTOR,N_EMPLOYEES,ECONOMIC_POT,ECO_POT_CLASS,RISK_CAT,YYYYMM,ITEM_ID,FLG_TOOL,SALES_CHANNEL,NET,UNIT,FAMILY_CODE,GROUP_CODE,CANCELLED,SALES_ID
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,P,XBXV2EB,XBXV2EB0102,NaN,8659.81_201709
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,P,XBXV2AB,XBXV2AB0102,NaN,8659.81_201709
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,P,XBXV2EB,XBXV2EB0102,NaN,8659.81_201709
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,P,XBXV1AF,XBXV1AF0201,NaN,8659.81_202010
4,939,2005-11-15,LE,15500,2,681.26,E,3d,2017-05-01,8241,0,B,64.51,P,XAES4AA,XAES4AA0101,X,681.26_201705


In [ ]:
# checking the number of unique sales per client
client_sales_count = df.groupby('CLIENT_ID')['SALES_ID'].nunique().sort_values(ascending=False)
print(client_sales_count)

CLIENT_ID
51708    60
9659     58
27926    58
19562    58
37126    57
         ..
50194     1
50199     1
50209     1
50219     1
93257     1
Name: SALES_ID, Length: 93257, dtype: int64


In [ ]:
# filtering the clients with more than one sale
client_sales_count.drop(client_sales_count[client_sales_count < 2].index, inplace=True)

pluri_client = df[df['CLIENT_ID'].isin(client_sales_count.index)]
print(pluri_client.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1931363 entries, 0 to 2048901
Data columns (total 18 columns):
 #   Column              Dtype         
---  ------              -----         
 0   CLIENT_ID           int64         
 1   CLIENT_CREATE DATE  datetime64[ns]
 2   REGION              object        
 3   TRADE SECTOR        int64         
 4   N_EMPLOYEES         int64         
 5   ECONOMIC_POT        float64       
 6   ECO_POT_CLASS       object        
 7   RISK_CAT            object        
 8   YYYYMM              datetime64[ns]
 9   ITEM_ID             int64         
 10  FLG_TOOL            int64         
 11  SALES_CHANNEL       object        
 12  NET                 float64       
 13  UNIT                object        
 14  FAMILY_CODE         object        
 15  GROUP_CODE          object        
 16  CANCELLED           object        
 17  SALES_ID            object        
dtypes: datetime64[ns](2), float64(2), int64(5), object(9)
memory usage: 280.0+ MB
None


We have now a new dataset cointaining only client with more than one purchase